<a href="https://colab.research.google.com/github/MuratCKoc/salesinsider/blob/main/ProphetsPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from fbprophet import Prophet
import pandas as pd
import numpy as np

In [18]:
df = pd.read_csv('https://raw.githubusercontent.com/MuratCKoc/salesinsider/main/salesinsider/data/Groceries_dataset.csv')
data_df = df[['Date', 'itemDescription']]
data_df.head()

,Date,itemDescription
0,21-07-2015,tropical fruit
1,05-01-2015,whole milk
2,19-09-2015,pip fruit
3,12-12-2015,other vegetables
4,01-02-2015,whole milk


In [19]:
cat_df = pd.get_dummies(data_df["itemDescription"])
final_df = pd.concat([data_df,cat_df],axis=1)
grouped_df = final_df.groupby("Date", as_index=False).sum()

grouped_df.head()

,Date,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,beverages,bottled beer,bottled water,brandy,brown bread,butter,butter milk,cake bar,candles,candy,canned beer,canned fish,canned fruit,canned vegetables,cat food,cereals,chewing gum,chicken,chocolate,chocolate marshmallow,citrus fruit,cleaner,cling film/bags,cocoa drinks,coffee,condensed milk,cooking chocolate,cookware,cream,...,salt,salty snack,sauces,sausage,seasonal products,semi-finished bread,shopping bags,skin care,sliced cheese,snack products,soap,soda,soft cheese,softener,soups,sparkling wine,specialty bar,specialty cheese,specialty chocolate,specialty fat,specialty vegetables,spices,spread cheese,sugar,sweet spreads,syrup,tea,tidbits,toilet cleaner,tropical fruit,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,01-01-2014,2,0,0,0,0,0,0,0,0,1,0,1,4,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,...,0,0,0,1,0,0,2,0,1,0,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,2,2,0,0,0,2,4,0
1,01-01-2015,0,0,0,0,0,0,0,0,1,0,0,1,1,0,2,2,0,0,0,0,3,0,0,0,0,0,0,0,1,0,3,0,0,0,0,0,0,0,0,...,1,1,0,2,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4,1,0,0,1,0,1,0,3,0,0
2,01-02-2014,0,0,0,0,0,0,0,0,2,0,0,2,1,0,3,2,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,4,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,2,1,0,1,0,1,3,0
3,01-02-2015,0,1,0,1,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,0,0,1,0,0,1,0,9,6,0
4,01-03-2014,0,0,0,0,0,0,0,0,0,1,0,1,2,0,2,0,1,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,2,1,0,2,0,0,0,0,5,0,0,1,0,0,0,1,0,0,2,0,1,0,0,0,0,0,0,0,0,1,3,0,1,0,1,1,0


In [29]:
columns = grouped_df.columns
print (columns)

Index(['Date', 'Instant food products', 'UHT-milk', 'abrasive cleaner',
       'artif. sweetener', 'baby cosmetics', 'bags', 'baking powder',
       'bathroom cleaner', 'beef',
       ...
       'turkey', 'vinegar', 'waffles', 'whipped/sour cream', 'whisky',
       'white bread', 'white wine', 'whole milk', 'yogurt', 'zwieback'],
      dtype='object', length=168)


In [36]:
master_df = pd.DataFrame()

In [ ]:
 for j in range(726, len(grouped_df.index)):
  for i in range (1, len(grouped_df.columns)): 
    prophet_df = grouped_df[["Date", grouped_df.columns[i]]]
    prophet_df.columns = ["ds", 'y']
    prophet_df['ds'] = pd.to_datetime(prophet_df.ds,format='%d-%m-%Y') 
    model = Prophet()
    model.fit(prophet_df)
    future = model.make_future_dataframe(periods=213)
    forecast = model.predict(future)
    predicted_df = forecast[["ds", "yhat"]]
    predicted_df.yhat = predicted_df.yhat.round()
    predicted_df.rename({})
    predicted_df = predicted_df.rename( columns={"ds": "Date", "yhat": grouped_df.columns[i]})
    print(predicted_df)
    if len(master_df.index) == 0:
      master_df = predicted_df.copy()
    else:
      master_df = pd.merge(master_df, predicted_df, on="Date")
    print(master_df)
    
  print(predicted_df)
  print(i)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWar

          Date  Instant food products
0   2014-01-01                    0.0
1   2014-01-02                    0.0
2   2014-01-03                    0.0
3   2014-01-04                    0.0
4   2014-01-05                    0.0
..         ...                    ...
936 2016-07-26                    0.0
937 2016-07-27                    0.0
938 2016-07-28                    0.0
939 2016-07-29                    0.0
940 2016-07-30                    0.0

[941 rows x 2 columns]
          Date  Instant food products_x  ...  frozen fruits  Instant food products_y
0   2014-01-01                      0.0  ...            0.0                      0.0
1   2014-01-02                      0.0  ...            0.0                      0.0
2   2014-01-03                      0.0  ...            0.0                      0.0
3   2014-01-04                      0.0  ...            0.0                      0.0
4   2014-01-05                      0.0  ...            0.0                      0.0
..        

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          Date  UHT-milk
0   2014-01-01       0.0
1   2014-01-02       0.0
2   2014-01-03       0.0
3   2014-01-04       0.0
4   2014-01-05       1.0
..         ...       ...
936 2016-07-26       0.0
937 2016-07-27       1.0
938 2016-07-28       0.0
939 2016-07-29       0.0
940 2016-07-30       0.0

[941 rows x 2 columns]
          Date  Instant food products_x  ...  Instant food products_y  UHT-milk_y
0   2014-01-01                      0.0  ...                      0.0         0.0
1   2014-01-02                      0.0  ...                      0.0         0.0
2   2014-01-03                      0.0  ...                      0.0         0.0
3   2014-01-04                      0.0  ...                      0.0         0.0
4   2014-01-05                      0.0  ...                      0.0         1.0
..         ...                      ...  ...                      ...         ...
936 2016-07-26                      0.0  ...                      0.0         0.0
937 2016-07-27      

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

          Date  margarine
0   2014-01-01        1.0
1   2014-01-02        1.0
2   2014-01-03        1.0
3   2014-01-04        1.0
4   2014-01-05        1.0
..         ...        ...
936 2016-07-26        0.0
937 2016-07-27        0.0
938 2016-07-28        0.0
939 2016-07-29        0.0
940 2016-07-30        0.0

[941 rows x 2 columns]
          Date  Instant food products_x  ...  male cosmetics  margarine
0   2014-01-01                      0.0  ...             0.0        1.0
1   2014-01-02                      0.0  ...             0.0        1.0
2   2014-01-03                      0.0  ...             0.0        1.0
3   2014-01-04                      0.0  ...             0.0        1.0
4   2014-01-05                      0.0  ...             0.0        1.0
..         ...                      ...  ...             ...        ...
936 2016-07-26                      0.0  ...             0.0        0.0
937 2016-07-27                      0.0  ...             0.0        0.0
938 2016-07-28  

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          Date  mayonnaise
0   2014-01-01         0.0
1   2014-01-02         0.0
2   2014-01-03         0.0
3   2014-01-04         0.0
4   2014-01-05         0.0
..         ...         ...
936 2016-07-26         0.0
937 2016-07-27         0.0
938 2016-07-28         0.0
939 2016-07-29         0.0
940 2016-07-30         0.0

[941 rows x 2 columns]
          Date  Instant food products_x  ...  margarine  mayonnaise
0   2014-01-01                      0.0  ...        1.0         0.0
1   2014-01-02                      0.0  ...        1.0         0.0
2   2014-01-03                      0.0  ...        1.0         0.0
3   2014-01-04                      0.0  ...        1.0         0.0
4   2014-01-05                      0.0  ...        1.0         0.0
..         ...                      ...  ...        ...         ...
936 2016-07-26                      0.0  ...        0.0         0.0
937 2016-07-27                      0.0  ...        0.0         0.0
938 2016-07-28                      0.0 

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          Date  meat
0   2014-01-01   0.0
1   2014-01-02   0.0
2   2014-01-03   0.0
3   2014-01-04   0.0
4   2014-01-05   0.0
..         ...   ...
936 2016-07-26   1.0
937 2016-07-27   1.0
938 2016-07-28   1.0
939 2016-07-29   1.0
940 2016-07-30   1.0

[941 rows x 2 columns]
          Date  Instant food products_x  ...  mayonnaise  meat
0   2014-01-01                      0.0  ...         0.0   0.0
1   2014-01-02                      0.0  ...         0.0   0.0
2   2014-01-03                      0.0  ...         0.0   0.0
3   2014-01-04                      0.0  ...         0.0   0.0
4   2014-01-05                      0.0  ...         0.0   0.0
..         ...                      ...  ...         ...   ...
936 2016-07-26                      0.0  ...         0.0   1.0
937 2016-07-27                      0.0  ...         0.0   1.0
938 2016-07-28                      0.0  ...         0.0   1.0
939 2016-07-29                      0.0  ...         0.0   1.0
940 2016-07-30                 